In [ ]:
import numpy as np

def compute_mean_class(X):
    """
    Compute the mean of each class in the dataset.
    """
    return np.mean(X, axis=0)

def compute_similarity(ac, class_attributes):
    """
    Compute the similarity between the unseen class attribute and each of the seen class attributes.
    """
    similarities = np.dot(ac, class_attributes.T)
    return similarities / np.sum(similarities)

def prototype_based_classifier(X_test, means):
    """
    Classify each test sample by finding the nearest mean.
    """
    num_test_samples = X_test.shape[0]
    predicted_labels = np.zeros(num_test_samples)

    for i in range(num_test_samples):
        distances = np.linalg.norm(means - X_test[i, :], axis=1)
        predicted_labels[i] = np.argmin(distances)+1

    return predicted_labels

# Load datasets
X_seen = np.load('/kaggle/input/x-seen/X_seen.npy', allow_pickle=True, encoding='bytes')
Xtest = np.load('/kaggle/input/x-test/Xtest.npy', allow_pickle=True, encoding='bytes')
Ytest = np.load('/kaggle/input/y-test/Ytest.npy', allow_pickle=True, encoding='bytes')
class_attributes_seen = np.load('/kaggle/input/class-attributes-seennpy/class_attributes_seen.npy', allow_pickle=True, encoding='bytes')
class_attributes_unseen = np.load('/kaggle/input/class-attributes-unseen/class_attributes_unseen.npy', allow_pickle=True, encoding='bytes')

# Method 1
means_seen = [compute_mean_class(X) for X in X_seen]
means_seen = np.vstack(means_seen)

means_unseen = []
for ac in class_attributes_unseen:
    similarity_vector = compute_similarity(ac, class_attributes_seen)
    mean_unseen = np.dot(similarity_vector, means_seen)
    means_unseen.append(mean_unseen)
means_unseen = np.vstack(means_unseen)

# Predictions
predicted_labels = prototype_based_classifier(Xtest, means_unseen)
accuracy = np.mean(predicted_labels == Ytest.ravel()) * 100
print(f"Method 1 Accuracy: {accuracy:.2f}%")


In [ ]:
import numpy as np

def compute_mean_class(X):
    """
    Compute the mean of each class in the dataset.
    """
    return np.mean(X, axis=0)

def prototype_based_classifier(X_test, means):
    """
    Classify each test sample by finding the nearest mean.
    """
    num_test_samples = X_test.shape[0]
    predicted_labels = np.zeros(num_test_samples)

    for i in range(num_test_samples):
        distances = np.linalg.norm(means - X_test[i, :], axis=1)
        predicted_labels[i] = np.argmin(distances)+1

    return predicted_labels

def train_multi_output_regression(As, Ms, lambda_val):
    """
    Train a multi-output regression model.
    """
    W = np.linalg.inv(As.T @ As + lambda_val * np.eye(As.shape[1])) @ As.T @ Ms
    return W

# Load datasets
X_seen = np.load('/kaggle/input/x-seen/X_seen.npy', allow_pickle=True, encoding='bytes')
Xtest = np.load('/kaggle/input/x-test/Xtest.npy', allow_pickle=True, encoding='bytes')
Ytest = np.load('/kaggle/input/y-test/Ytest.npy', allow_pickle=True, encoding='bytes')
class_attributes_seen = np.load('/kaggle/input/class-attributes-seennpy/class_attributes_seen.npy', allow_pickle=True, encoding='bytes')
class_attributes_unseen = np.load('/kaggle/input/class-attributes-unseen/class_attributes_unseen.npy', allow_pickle=True, encoding='bytes')

# Method 2
means_seen = [compute_mean_class(X) for X in X_seen]
means_seen = np.vstack(means_seen)

# Train the multi-output regression model
lambdas = [0.01, 0.1, 1, 10, 20, 50, 100]
best_lambda = None
best_accuracy = 0

for lambda_val in lambdas:
    W = train_multi_output_regression(class_attributes_seen, means_seen, lambda_val)
    means_unseen = class_attributes_unseen @ W

    # Predictions
    predicted_labels = prototype_based_classifier(Xtest, means_unseen)
    accuracy = np.mean(predicted_labels == Ytest.ravel()) * 100
    print(f"Method 2 Accuracy with lambda = {lambda_val}: {accuracy:.2f}%")

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_lambda = lambda_val

print(f"Best lambda: {best_lambda}, Best Accuracy: {best_accuracy:.2f}%")
